# Recurrent Neural Networks

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 03/03/25  | Martin | Created   | Skipped Sentiment Classification section, completed Stock price prediction section. Stopped working on this because the dataset is too large | 

# Content

* [Open-domain Question Answering](#open-domain-question-answering)

# Open-domain Question Answering

_Question-answering (QA):_ Emulate the human process of searching for information online, with machine learning methods employed to improve accuracy of the provided answers

🎯 Predict long and short responses to questions about Wikipedia articles

For each article-question pair -> predict long- and short-form answers to the question drawn directly from the article:

* A __long answer__ would be a longer section of text that answers the question - several sentences or a paragraph
* A __short answer__ might be a sentence or phrase. Always contained within, or a subset of, one of the plausible long answers
* Mostly articles can have both long and short answers

In [ ]:
import os
import json
import gc
import pickle

import numpy as np
import pandas as pd
import fasttext

from tqdm import tqdm_notebook as tqdm

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, Masking
from tensorflow.keras.layers import  LSTM, Bidirectional, GlobalMaxPooling1D, Dropout
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import load_model

2025-03-03 17:27:04.447785: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-03 17:27:06.376809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740994027.170684    3549 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740994027.377090    3549 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 17:27:08.880953: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [9]:
def build_train(
  train_path: str, 
  n_rows: int=200000,
  sampling_rate: int=15
):
  """
  Creates the training data by performing data preprocessing to separate the long answers,
  short answers and annotations.

  Args:
      train_path (str): path to raw data text
      n_rows (int, optional): Number of training samples to create. Defaults to 200000.
      sampling_rate (int, optional): _description_. Defaults to 15.
  
  Returns:
      pd.DataFrame: training dataframe
  """
  with open(train_path) as f:
    processed_rows = []

    for i in tqdm(range(n_rows)):
      line = f.readline()
      if not line:
        break

      # Each line is a json object, so we load it as is
      line = json.loads(line)

      text = line['document_text'].split(' ') # separate into words
      question = line['question_text']
      annotations = line['annotations'][0]

      for i, candidate in enumerate(line['long_answer_candidate']):
        label = i == annotations['long_answer']['candidate_index']

        start = candidate['start_token']
        end = candidate['end_token']

        if label or (i % sampling_rate == 0):
          processed_rows.append({
            'text': " ".join(text[start:end]),
            'is_long_answer': label,
            'question': question,
            'annotation_id': annotations['annotation_id']
          })
    
    train = pd.DataFrame(processed_rows)

    return train

Due to size of dataframe, stopped working on this recipe